論文<br>
https://arxiv.org/abs/2302.07483<br>
<br>
GitHub<br>
https://github.com/LSH9832/edgeyolo<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/EdgeYOLO_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# setup environment

## check gpu

In [ ]:
!nvidia-smi

## clone code from github

In [ ]:
%cd /content

!git clone https://github.com/LSH9832/edgeyolo.git

%cd /content/edgeyolo
# Commits on Feb 28, 2023
!git checkout 673e270917e4db45967b6106585339dd8d7913dd

## install libraries

In [ ]:
%cd /content/edgeyolo

# Install dependent packages
!pip install -r requirements.txt

!pip install moviepy==0.2.3.5 imageio==2.4.1

## import libraries

In [ ]:
%cd /content/edgeyolo

import os
import time
from datetime import datetime as date
from glob import glob

import cv2
from moviepy.video.fx.resize import resize
from moviepy.editor import VideoFileClip

from edgeyolo.detect import Detector
from detect import draw

# setup pretrained models

In [ ]:
%cd /content/edgeyolo

# create directory
!mkdir ./pretrained

# download model
!wget -c https://github.com/LSH9832/edgeyolo/releases/download/v0.0.0/edgeyolo_coco.pth \
      -O ./pretrained/edgeyolo_coco.pth

# setup test videos

In [ ]:
%cd /content/edgeyolo

# create directory
!mkdir ./test_video

!wget -c https://github.com/LSH9832/edgeyolo/releases/download/v0.0.0/test.avi \
      -O ./test_video/test.avi

# Object detection

In [ ]:
WEIGHTS = './pretrained/edgeyolo_coco.pth'
SOURCE = './test_video/test.avi' # or dir with images, such as /dataset/coco2017/val2017 (jpg/jpeg, png, bmp, webp is available)
CONF_THRES = 0.25
NMS_THRES = 0.5
INPUT_SIZE = [640, 640] # height width
BATCH = 1
SAVE_DIR = './output'
NO_LABEL = False

In [ ]:
%cd /content/edgeyolo

# create detecter
detector = Detector 
detect = detector(
    weight_file = WEIGHTS,
    conf_thres = CONF_THRES,
    nms_thres = NMS_THRES,
    input_size = INPUT_SIZE,
    fuse = True,
    fp16 = False,
    use_decoder = False
    )

# load video
source = cv2.VideoCapture(SOURCE)
fps = source.get(cv2.CAP_PROP_FPS)
print("fps:", fps)

all_dt = []
dts_len = 300 // BATCH
success = True

# start inference
count = 0
t_start = time.time()
while source.isOpened() and success:
  frames = []
  for _ in range(BATCH):
    success, frame = source.read()
    if not success:
      if not len(frames):
        cv2.destroyAllWindows()
        break
      else:
        while len(frames) < BATCH:
          frames.append(frames[-1])
    else:
      frames.append(frame)

  if not len(frames):
    break

  # Inference
  results = detect(frames, False)

  dt = detect.dt
  all_dt.append(dt)
  if len(all_dt) > dts_len:
    all_dt = all_dt[-dts_len:]
  print(f"\r{dt * 1000 / BATCH:.1f}ms  "
    f"average:{sum(all_dt) / len(all_dt) / BATCH * 1000:.1f}ms", end="      ")
  
  # draw detect result
  imgs = draw(frames, results, detect.class_names, 2, draw_label=not NO_LABEL)

  for img in imgs:
    # create save dir
    os.makedirs(SAVE_DIR, exist_ok=True)
    file_name = f"{str(date.now()).split('.')[0].replace(':', '').replace('-', '').replace(' ', '')}.jpg"
    cv2.imwrite(os.path.join(SAVE_DIR, file_name), img)

source.release()

In [ ]:
det_frames = glob( os.path.join(SAVE_DIR, '*.jpg'))
det_frames.sort()
print('num of frames:', len(det_frames))

h, w, _ = cv2.imread(det_frames[0]).shape

# jpg to mp4
video = cv2.VideoWriter(
    os.path.join(SAVE_DIR, 'video.mp4'),
    fourcc=cv2.VideoWriter_fourcc(*'mp4v'),
    fps=float(fps),
    frameSize=(w, h),
    isColor=True)

for frame_path in det_frames:
  img = cv2.imread(frame_path)
  video.write(img)

print(video.get(cv2.CAP_PROP_FPS), (INPUT_SIZE[0], INPUT_SIZE[1]))
video.release()

In [ ]:
# show video
clip = VideoFileClip(os.path.join(SAVE_DIR, 'video.mp4'))
clip = resize(clip, height=420)
clip.ipython_display()